## 1.导入依赖以及网络结构

In [72]:
# 加载网络结构
%run AI2Flutter.py

## 2.实例化网络，并设置模型输入形状

In [74]:
# 超参数
num_layers = 4
d_model = 256
dff = 512
num_heads = 8
dropout_rate = 0.1
input_node_dim = 24  # demo中均使用24维向量
target_node_dim = 24  # 

# 权重保存位置
save_weight_path = "./model_weight/model_1"
save_path = "./model/model_1"

transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_node_dim=input_node_dim,
    target_node_dim=target_node_dim,
    dropout_rate=dropout_rate)

# 优化器采用Adam，学习率自定义
learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)
transformer.compile(
    loss='mean_squared_error',
    optimizer=optimizer,
)
# 设置模型输入形状
transformer((tf.keras.layers.Input(shape=(None, input_node_dim,)),
             tf.keras.layers.Input(shape=(None, target_node_dim,))))
# 网络概览
transformer.summary()

Model: "transformer_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_13 (Encoder)        multiple                  2642048   
                                                                 
 decoder_13 (Decoder)        multiple                  4753024   
                                                                 
 dense_133 (Dense)           multiple                  3096      
                                                                 
Total params: 7,398,168
Trainable params: 7,398,168
Non-trainable params: 0
_________________________________________________________________


## 3.加载已经训练的权重，方便继续训练

In [3]:
# 加载已训练权重
transformer.load_weights(save_weight_path)

## 4.加载数据集训练网络

In [75]:
%run AI2Flutter_demo_data.py
# 数据规模
train_seqs_num = 200
validation_seqs_num = 30
# 生成随机数据集
input_data, output_data, output_label = demo_generate_data(train_seqs_num)
vali_input_data, vali_output_data, vali_output_label = demo_generate_data(validation_seqs_num)

# 训练集
train_input = tf.data.Dataset.from_generator(
    lambda: input_data, 
    output_signature=(
        tf.TensorSpec(shape=(None, input_node_dim), dtype=tf.float32)))
train_output = tf.data.Dataset.from_generator(
    lambda: output_data, 
    output_signature=(
        tf.TensorSpec(shape=(None, target_node_dim), dtype=tf.float32)))
train_label = tf.data.Dataset.from_generator(
    lambda: output_label, 
    output_signature=(
        tf.TensorSpec(shape=(None, target_node_dim), dtype=tf.float32)))
train_dataset = tf.data.Dataset.zip(((train_input, train_output), train_label))
# batch设置为4
train_dataset = train_dataset.padded_batch(4)

# 验证集
vali_input = tf.data.Dataset.from_generator(
    lambda: vali_input_data, 
    output_signature=(
        tf.TensorSpec(shape=(None, input_node_dim), dtype=tf.float32)))
vali_output = tf.data.Dataset.from_generator(
    lambda: vali_output_data, 
    output_signature=(
        tf.TensorSpec(shape=(None, target_node_dim), dtype=tf.float32)))
vali_label = tf.data.Dataset.from_generator(
    lambda: vali_output_label, 
    output_signature=(
        tf.TensorSpec(shape=(None, target_node_dim), dtype=tf.float32)))
vali_dataset = tf.data.Dataset.zip(((vali_input, vali_output), vali_label))
vali_dataset = vali_dataset.padded_batch(1)

In [76]:
# 训练
transformer.fit(
    x=train_dataset,
    epochs=500,
#     validation_data=train_dataset
)

Epoch 1/500
50/50 [==============================] - 8s 28ms/step - loss: 47439760.0000
Epoch 2/500
50/50 [==============================] - 1s 27ms/step - loss: 47431936.0000
Epoch 3/500
50/50 [==============================] - 1s 29ms/step - loss: 47426964.0000
Epoch 4/500
50/50 [==============================] - 2s 31ms/step - loss: 47424436.0000
Epoch 5/500
50/50 [==============================] - 1s 26ms/step - loss: 47422808.0000
Epoch 6/500
50/50 [==============================] - 1s 28ms/step - loss: 47421496.0000
Epoch 7/500
50/50 [==============================] - 1s 29ms/step - loss: 47420088.0000
Epoch 8/500
50/50 [==============================] - 1s 27ms/step - loss: 47418428.0000
Epoch 9/500
50/50 [==============================] - 1s 27ms/step - loss: 47416536.0000
Epoch 10/500
50/50 [==============================] - 1s 27ms/step - loss: 47414268.0000
Epoch 11/500
50/50 [==============================] - 1s 26ms/step - loss: 47411728.0000
Epoch 12/500
50/50 [==========

50/50 [==============================] - 1s 27ms/step - loss: 41868812.0000
Epoch 94/500
50/50 [==============================] - 1s 26ms/step - loss: 41781388.0000
Epoch 95/500
50/50 [==============================] - 1s 26ms/step - loss: 41697908.0000
Epoch 96/500
50/50 [==============================] - 1s 26ms/step - loss: 41618312.0000
Epoch 97/500
50/50 [==============================] - 1s 26ms/step - loss: 41542120.0000
Epoch 98/500
50/50 [==============================] - 1s 26ms/step - loss: 41469756.0000
Epoch 99/500
50/50 [==============================] - 1s 27ms/step - loss: 41433524.0000
Epoch 100/500
50/50 [==============================] - 1s 27ms/step - loss: 41369456.0000
Epoch 101/500
50/50 [==============================] - 1s 27ms/step - loss: 41276000.0000
Epoch 102/500
50/50 [==============================] - 1s 27ms/step - loss: 41217960.0000
Epoch 103/500
50/50 [==============================] - 1s 27ms/step - loss: 41163484.0000
Epoch 104/500
50/50 [=========

50/50 [==============================] - 1s 26ms/step - loss: 40022976.0000
Epoch 185/500
50/50 [==============================] - 1s 26ms/step - loss: 40256300.0000
Epoch 186/500
50/50 [==============================] - 1s 26ms/step - loss: 40239028.0000
Epoch 187/500
50/50 [==============================] - 1s 27ms/step - loss: 40168040.0000
Epoch 188/500
50/50 [==============================] - 1s 26ms/step - loss: 40037256.0000
Epoch 189/500
50/50 [==============================] - 1s 26ms/step - loss: 40060972.0000
Epoch 190/500
50/50 [==============================] - 1s 27ms/step - loss: 40017280.0000
Epoch 191/500
50/50 [==============================] - 1s 26ms/step - loss: 40056492.0000
Epoch 192/500
50/50 [==============================] - 1s 27ms/step - loss: 40012352.0000
Epoch 193/500
50/50 [==============================] - 1s 26ms/step - loss: 39920920.0000
Epoch 194/500
50/50 [==============================] - 1s 27ms/step - loss: 39987212.0000
Epoch 195/500
50/50 [===

50/50 [==============================] - 1s 26ms/step - loss: 37586200.0000
Epoch 276/500
50/50 [==============================] - 1s 27ms/step - loss: 37678496.0000
Epoch 277/500
50/50 [==============================] - 1s 26ms/step - loss: 37587000.0000
Epoch 278/500
50/50 [==============================] - 1s 26ms/step - loss: 37575564.0000
Epoch 279/500
50/50 [==============================] - 1s 27ms/step - loss: 37476980.0000
Epoch 280/500
50/50 [==============================] - 1s 27ms/step - loss: 37521996.0000
Epoch 281/500
50/50 [==============================] - 1s 28ms/step - loss: 37522856.0000
Epoch 282/500
50/50 [==============================] - 1s 26ms/step - loss: 37492880.0000
Epoch 283/500
50/50 [==============================] - 1s 26ms/step - loss: 37429452.0000
Epoch 284/500
50/50 [==============================] - 1s 26ms/step - loss: 37356464.0000
Epoch 285/500
50/50 [==============================] - 1s 26ms/step - loss: 37404168.0000
Epoch 286/500
50/50 [===

50/50 [==============================] - 1s 28ms/step - loss: 35536312.0000
Epoch 367/500
50/50 [==============================] - 1s 28ms/step - loss: 36092180.0000
Epoch 368/500
50/50 [==============================] - 1s 29ms/step - loss: 35898812.0000
Epoch 369/500
50/50 [==============================] - 1s 27ms/step - loss: 36252332.0000
Epoch 370/500
50/50 [==============================] - 1s 29ms/step - loss: 35757292.0000
Epoch 371/500
50/50 [==============================] - 2s 36ms/step - loss: 35832388.0000
Epoch 372/500
50/50 [==============================] - 1s 29ms/step - loss: 35735528.0000
Epoch 373/500
50/50 [==============================] - 1s 29ms/step - loss: 36037764.0000
Epoch 374/500
50/50 [==============================] - 1s 28ms/step - loss: 35893352.0000
Epoch 375/500
50/50 [==============================] - 1s 27ms/step - loss: 35845352.0000
Epoch 376/500
50/50 [==============================] - 1s 30ms/step - loss: 35892156.0000
Epoch 377/500
50/50 [===

50/50 [==============================] - 1s 27ms/step - loss: 34826592.0000
Epoch 458/500
50/50 [==============================] - 1s 27ms/step - loss: 35568020.0000
Epoch 459/500
50/50 [==============================] - 1s 27ms/step - loss: 34574580.0000
Epoch 460/500
50/50 [==============================] - 1s 27ms/step - loss: 35440360.0000
Epoch 461/500
50/50 [==============================] - 1s 27ms/step - loss: 35258848.0000
Epoch 462/500
50/50 [==============================] - 1s 27ms/step - loss: 35140120.0000
Epoch 463/500
50/50 [==============================] - 1s 27ms/step - loss: 34833368.0000
Epoch 464/500
50/50 [==============================] - 1s 27ms/step - loss: 34877748.0000
Epoch 465/500
50/50 [==============================] - 1s 27ms/step - loss: 34743024.0000
Epoch 466/500
50/50 [==============================] - 1s 27ms/step - loss: 34636284.0000
Epoch 467/500
50/50 [==============================] - 1s 27ms/step - loss: 34758796.0000
Epoch 468/500
50/50 [===

## 5.使用网络预测

In [79]:
# # 预测，基于原先的全是1的样本，网络似乎学会了把任何数字全部映射为1
# input1 = tf.zeros((1,5,24))
# input2 = tf.constant([[[
#     12526,-52,-97877773352,45,5,6,7,8,9,10,-22099995,-12,13,999,15,167866687,17,18,19,10,0,0,0,0
# ]]])
# # input2 = tf.ones((1, 1, 24))
# # input2 = tf.zeros((1, 1, 24))
# input2 = tf.cast(input2, tf.float32)
# re = transformer((input1, input2), training=False)
# print(input2)
# print(re)
# print(tf.reduce_sum(tf.abs(re - tf.ones((1, 1, 24)))))

input1, input2, output = demo_generate_data(1)
start = [[[-1 for i in range(output_seq_dim)]]]
p1 = transformer((tf.constant(input1), tf.constant(start)), training=False)
print(p1)
print(output[0][0])
print("***************")
d = np.array(p1 - output[0][0])
print(d)
print(np.sum(d**2))

tf.Tensor(
[[[ 1.3431368e+00  1.3543750e+00 -1.9080779e-01  1.2191708e+04
    1.4264363e+04  8.2335645e+03  7.9567627e+03  2.2426299e+03
    2.4998171e+03  4.0127897e+00  7.7040659e+03  8.9238076e+03
    3.3457300e+03  3.1872917e+03  1.4882181e+03  1.9194377e+03
    2.3226509e+01  8.5134983e+00  8.9365416e+00 -1.8620905e-01
   -1.8570238e-01 -1.8455765e-01 -1.8512613e-01 -1.8593711e-01]]], shape=(1, 1, 24), dtype=float32)
[2, 2, 1, 33521, 38596, 32852, 30431, 25163, 28216, 0, 27979, 35797, 100, 101, 109, 111, 0, 25, 29, 0, 0, 0, 0, 0]
***************
[[[-6.5686321e-01 -6.4562500e-01 -1.1908078e+00 -2.1329293e+04
   -2.4331637e+04 -2.4618436e+04 -2.2474238e+04 -2.2920371e+04
   -2.5716184e+04  4.0127897e+00 -2.0274934e+04 -2.6873191e+04
    3.2457300e+03  3.0862917e+03  1.3792181e+03  1.8084377e+03
    2.3226509e+01 -1.6486502e+01 -2.0063457e+01 -1.8620905e-01
   -1.8570238e-01 -1.8455765e-01 -1.8512613e-01 -1.8593711e-01]]]
4503267000.0


## 6.保存模型的权重，方便下一次训练

In [15]:
#保存训练权重
transformer.save_weights(save_weight_path)

## 7.保存整个模型，方便迁移到其他地方

In [16]:
# 直接加载模型
# transformer = tf.saved_model.load("model2")
# 保存模型
tf.saved_model.save(transformer, save_path)

INFO:tensorflow:Assets written to: ./model/model_1/assets


INFO:tensorflow:Assets written to: ./model/model_1/assets
